In [37]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [38]:
df = pd.read_parquet('data/fhv_tripdata_2021-01.parquet')
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [39]:
# Q1
len(df)

1154112

In [40]:
# Q2
df['duration'] = df['dropOff_datetime'] - df['pickup_datetime']
df['duration'].mean().total_seconds()/60

19.167224083333334

In [41]:
# Data cleaning
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [42]:
# Q3
categorical = ['PUlocationID', 'DOlocationID']
X_train = df[categorical].fillna(-1)

In [43]:
sum(X_train['PUlocationID'] == -1)/len(df)

0.8352732770722617

In [44]:
# Q4
X_train = pd.get_dummies(X_train)

In [46]:
X_train[categorical] = X_train[categorical].astype(str)

In [47]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1109826 entries, 0 to 1154111
Data columns (total 2 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   PUlocationID  1109826 non-null  object
 1   DOlocationID  1109826 non-null  object
dtypes: object(2)
memory usage: 25.4+ MB


In [50]:
# Q5
y_train = df['duration'].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

11.415432830521663

In [56]:
# Q6
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df['dropOff_datetime'] - df['pickup_datetime']
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [57]:
df_train = read_dataframe('data/fhv_tripdata_2021-01.parquet')
df_val = read_dataframe('data/fhv_tripdata_2021-02.parquet')

In [58]:
df_train['PU_DO'] = df_train['PUlocationID'] + '_' + df_train['DOlocationID']
df_val['PU_DO'] = df_val['PUlocationID'] + '_' + df_val['DOlocationID']

In [60]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [61]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [63]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

10.596293717089068